In [2]:
import pandas as pd

# CHANGE THIS to your SAS07 folder (must contain 81sas07ews_0.csv ... 81sas07ews_4.csv)
base = "https://raw.githubusercontent.com/jourdee-lab/manchester-spatial-analysis-data-lab/1e2a6499d4f7997c50b0a150b60d7ab3e8857fde/data/raw/census_1981/sas07_employment"
files = [f"{base}/81sas07ews_{i}.csv" for i in range(5)]

dfs = [pd.read_csv(url, dtype={"zoneid": "string"}) for url in files]
sas07_1981 = pd.concat(dfs, ignore_index=True)
sas07_1981.columns = sas07_1981.columns.str.strip()
sas07_1981["zoneid"] = sas07_1981["zoneid"].str.strip()

value_cols = [c for c in sas07_1981.columns if c != "zoneid"]
sas07_1981[value_cols] = sas07_1981[value_cols].apply(pd.to_numeric, errors="coerce")

# Specify Manchester district
MANCHESTER_DISTRICT = "03BN"
manchester_zones = sas07_1981.loc[
    sas07_1981["zoneid"].str.startswith(MANCHESTER_DISTRICT),
    "zoneid"
].unique()

print(f"Found {len(manchester_zones)} Manchester zones")
print(f"Sample zones: {sorted(manchester_zones)[:5]}")

# Aggregate all Manchester zones
manchester_agg = pd.DataFrame(
    [sas07_1981.loc[sas07_1981["zoneid"].str.startswith(MANCHESTER_DISTRICT), value_cols].sum(numeric_only=True)],
    columns=value_cols
)

manchester_agg.insert(0, "zoneid", MANCHESTER_DISTRICT)

print("\n=== Manchester City (1981) - SAS07 Employment ===")
print(manchester_agg)
first_metric = next((c for c in value_cols if c.startswith("81sas07")), None)
if first_metric:
    print(f"\nExample check ({first_metric}): {int(manchester_agg[first_metric].iloc[0]):,}")

manchester_agg.to_csv("sas07_1981_combined.csv", index=False)
print("\nFull aggregate saved to: sas07_1981_combined.csv")

Found 1053 Manchester zones
Sample zones: ['03BN', '03BNAA', '03BNAA01', '03BNAA02', '03BNAA03']

=== Manchester City (1981) - SAS07 Employment ===
  zoneid  81sas070615  81sas070616  81sas070617  81sas070618  81sas070619  \
0   03BN       290496        95397       125604        14049         5058   

   81sas070620  81sas070621  81sas070622  81sas070623  ...  81sas070632  \
0          646        60219        13399        16084  ...         1690   

   81sas070633  81sas070634  81sas070635  81sas070636  81sas070637  \
0       263483        93375       120630       256029        77824   

   81sas070638  81sas070639  81sas070640  81sas070641  
0        56987         7454        15551        63643  

[1 rows x 28 columns]

Example check (81sas070615): 290,496

Full aggregate saved to: sas07_1981_combined.csv
